In [1]:
dude_ids = ['aa2ar', 'abl1', 'ace', 'aces', 'ada', 'ada17', 'adrb1', 'adrb2', 'akt1', 'akt2', 'aldr', 'ampc', 'andr', 'aofb', 'bace1', 'braf', 'cah2', 'casp3', 'cdk2', 'comt', 'cp2c9', 'cp3a4', 'csf1r', 'cxcr4', 'def', 'dhi1', 'dpp4', 'drd3', 'dyr', 'egfr', 'esr1', 'esr2', 'fa10', 'fa7', 'fabp4', 'fak1', 'fgfr1', 'fkb1a', 'fnta', 'fpps', 'gcr', 'glcm', 'gria2', 'grik1', 'hdac2', 'hdac8', 'hivint', 'hivpr', 'hivrt', 'hmdh', 'hs90a', 'hxk4', 'igf1r', 'inha', 'ital', 'jak2', 'kif11', 'kit', 'kith', 'kpcb', 'lck', 'lkha4', 'mapk2', 'mcr', 'met', 'mk01', 'mk10', 'mk14', 'mmp13', 'mp2k1', 'nos1', 'nram', 'pa2ga', 'parp1', 'pde5a', 'pgh1', 'pgh2', 'plk1', 'pnph', 'ppara', 'ppard', 'pparg', 'prgr', 'ptn1', 'pur2', 'pygm', 'pyrd', 'reni', 'rock1', 'rxra', 'sahh', 'src', 'tgfr1', 'thb', 'thrb', 'try1', 'tryb1', 'tysy', 'urok', 'vgfr2', 'wee1', 'xiap']
dude_ids = ['lck', 'src', 'ada17', 'hivpr', 'mk14', 'mmp13', 'aa2ar', 'bace1', 'pparg', 'parp1', 'ace', 'thrb', 'cdk2', 'esr1', 'esr2', 'vgfr2', 'fnta', 'drd3', 'csf1r', 'dhi1', 'casp3', 'gria2', 'kit', 'dyr', 'braf', 'tryb1', 'hdac8', 'aldr', 'akt1', 'ital', 'kpcb', 'tysy', 'ppard', 'hivint', 'ppara', 'urok', 'wee1', 'reni', 'grik1', 'aces', 'fa10', 'dpp4', 'adrb2', 'jak2', 'hivrt', 'fkb1a', 'cah2', 'kif11', 'try1', 'adrb1', 'akt2', 'rock1', 'pa2ga', 'pygm', 'mapk2', 'fa7', 'tgfr1', 'mk10', 'fak1', 'gcr', 'hdac2', 'prgr', 'ptn1', 'nram', 'abl1', 'hs90a', 'egfr', 'hxk4', 'mk01', 'cxcr4', 'lkha4', 'ada', 'pur2', 'pnph', 'andr', 'rxra', 'fpps', 'cp3a4', 'met', 'ampc', 'mp2k1', 'pyrd', 'pgh1', 'kith', 'thb', 'comt', 'cp2c9', 'aofb', 'fabp4', 'mcr', 'inha', 'pgh2', 'def', 'xiap', 'glcm', 'pde5a', 'nos1', 'sahh', 'hmdh', 'igf1r', 'plk1', 'fgfr1']

In [2]:
import pandas as pd

In [3]:
def train_rf(dude_id, engine = 'dude', v = 2):
    import numpy as np
    import pandas as pd
    from os.path import isfile
    from scipy.stats import pearsonr, spearmanr, kendalltau
    
    from sklearn.cross_validation import KFold, StratifiedKFold
    from sklearn.metrics import accuracy_score, precision_score, mean_squared_error, recall_score
    from sklearn.utils import compute_sample_weight
    
    import oddt
    from oddt.metrics import roc_auc, roc_log_auc, enrichment_factor
    from oddt.scoring.models.regressors import randomforest
    #from oddt.scoring.models.classifiers import randomforest
    from oddt.scoring import cross_validate
    
    oddt.random_seed(0)
    out = []
        
    act_cutoff = 6.
    inactive_cutoff = 5.95
    
    if v == 1:
        col_range = range(1,37)
        np_type = np.uint16
    elif v == 2:
        col_range = range(1,217)
        np_type = np.uint16
    elif v == 3:
        col_range = range(1,43)   
        np_type = np.float16
    
    # trap errors on reading
    try:
        actives_full = pd.read_csv('head1_full/%s/%s/%s_desc_v%i.csv.gz' % (dude_id, engine, 'actives', v), dtype={i: np_type for i in col_range})
        decoys_full = pd.read_csv('head1_full/%s/%s/%s_desc_v%i.csv.gz' % (dude_id, engine, 'decoys', v), dtype={i: np_type for i in col_range})
    except:
        return None
    
    #trap empty ones
    if len(actives_full) == 0 or len(decoys_full) == 0:
        return None
    
    decoys_full['act'] = inactive_cutoff if act_cutoff > 10 else 10**(9-inactive_cutoff)
    
    # generate one big table for dude_id
    data = pd.concat((actives_full, decoys_full))
    
    # normalize
    if act_cutoff >= 10:
        data['act'] = np.clip(data['act'], 1e-9, inactive_cutoff)
    else:
        data['act'] = np.clip(-np.log10(np.clip(data['act'], 1e-9, 1e9) * 1e-9), 0 , 15)
    # binary
    data['act_bin'] = data['act'] < act_cutoff if act_cutoff > 10 else data['act'] > act_cutoff# 1e-6 # close to zero

    # manual CV, use Stratified for even distribution
    oddt.random_seed(0)
    cv_split = StratifiedKFold(data['act_bin'], n_folds=5, shuffle=True)

    mtry=None
    if v == 2:
        mtry = 100
    else:
        mtry = 15

    oddt.random_seed(0)
    rf = randomforest(n_estimators=500, n_jobs=-1, verbose=1,
                        max_features=mtry, 
                        oob_score=True,
                        bootstrap = True,
                        random_state = 0,
                        )

    for train, test in cv_split:
        data_train = data.iloc[train]
        data_test = data.iloc[test]
        
        oddt.random_seed(0)
        rf.fit(data_train[col_range],
               data_train['act'],
               sample_weight=compute_sample_weight('balanced', data_train['act_bin']))
    
        data_test['pred'] = rf.predict(data_test[col_range])
        data_test = data_test.sort_values('pred', ascending=(act_cutoff >= 10))# log is descending
        if act_cutoff >= 10:
            data_test['pred_bin'] = data_test['pred'] < act_cutoff
        else:
            data_test['pred_bin'] = data_test['pred'] > act_cutoff

        try:
            # report only tests
            out.append({ 'dude_id': dude_id, 'engine': engine, 'v': v,
                'rp': pearsonr(data_test['act'], data_test['pred'])[0],
                'rs': spearmanr(data_test['act'], data_test['pred'])[0],
                'rk': kendalltau(data_test['act'], data_test['pred'])[0],

                'rp_active': pearsonr(data_test['act'][data_test['act_bin']], data_test['pred'][data_test['act_bin']])[0],
                'rs_active': spearmanr(data_test['act'][data_test['act_bin']], data_test['pred'][data_test['act_bin']])[0],
                'rk_active': kendalltau(data_test['act'][data_test['act_bin']], data_test['pred'][data_test['act_bin']])[0],

                'rp_inactive': pearsonr(data_test['act'][~data_test['act_bin']], data_test['pred'][~data_test['act_bin']])[0],
                'rs_inactive': spearmanr(data_test['act'][~data_test['act_bin']], data_test['pred'][~data_test['act_bin']])[0],
                'rk_inactive': kendalltau(data_test['act'][~data_test['act_bin']], data_test['pred'][~data_test['act_bin']])[0],

                'mse': mean_squared_error(data_test['act'], data_test['pred']),
                'roc_auc': roc_auc(data_test['act_bin'], data_test['pred_bin'], ascending_score=False),# Binary is descending
                'roc_log_auc': roc_log_auc(data_test['act_bin'], data_test['pred_bin'], ascending_score=False),# Binary is descending

                'ef0.1': enrichment_factor(data_test['act_bin'], data_test['pred_bin'], percentage=0.1),
                'ef1': enrichment_factor(data_test['act_bin'], data_test['pred_bin']),
                'ef2': enrichment_factor(data_test['act_bin'], data_test['pred_bin'], percentage=2),
                'ef5': enrichment_factor(data_test['act_bin'], data_test['pred_bin'], percentage=5),
                'ef10': enrichment_factor(data_test['act_bin'], data_test['pred_bin'], percentage=10),
                        
                'precision': precision_score(data_test['act_bin'], data_test['pred_bin']),
                'accuracy': accuracy_score(data_test['act_bin'], data_test['pred_bin']),
                'recall': recall_score(data_test['act_bin'], data_test['pred_bin']),
              })
        except:
            return None
    return pd.DataFrame(out)

In [4]:
#train_rf('cxcr4')

In [5]:
from cluster_helper.cluster import cluster_view

In [6]:
%%time
with cluster_view(scheduler="slurm", queue='long', num_jobs=20, cores_per_job=11) as view:
#with cluster_view(scheduler=None, queue=None, num_jobs=5, extra_params={"run_local": True}) as view:
    list_args = [(dude_id, engine, v) for dude_id in dude_ids for v in [1,2,3] for engine in ['dude','vina','dock']]
    args = zip(*list_args)
    o = view.map(train_rf, args[0],args[1],args[2])
res = pd.concat(o)
res.to_csv('dude_per_target.csv')

20 Engines running
Sending a shutdown signal to the controller and engines.
CPU times: user 23.3 s, sys: 2.46 s, total: 25.8 s
Wall time: 16min 32s


In [7]:
res.groupby(('engine', 'v')).mean()

accuracy      ef0.1        ef1      ef10        ef2        ef5  \
engine v                                                                   
dock   1  0.916979  60.254391  31.112807  6.728576  20.856623  11.208007   
       2  0.911882  71.453545  36.995909  7.232252  23.798486  12.406140   
       3  0.918412  71.430826  36.714012  7.321583  23.838323  12.554673   
dude   1  0.906239  52.284392  36.115826  7.711092  25.163880  13.330750   
       2  0.902079  57.849490  43.035093  8.182333  29.430834  14.691891   
       3  0.911839  56.619809  42.266753  8.231427  28.947586  14.658588   
vina   1  0.878581  49.507839  31.096563  7.326063  22.119075  12.143214   
       2  0.867875  52.577953  35.987770  7.757484  25.194533  13.349876   
       3  0.882805  52.912722  35.576579  7.809236  24.948722  13.378177   

               mse  precision    recall        rk  rk_active  rk_inactive  \
engine v                                                                    
dock   1  0.048174   0.101822  0.644631  0.115178   0.135700          NaN   
       2  0.045789   0.103499  0.701967  0.122790   0.155949          NaN   
       3  0.044985   0.114629  0.707055  0.124569   0.174584          NaN   
dude   1  0.041204   0.139388  0.776199  0.150337   0.200439          NaN   
       2  0.037291   0.148497  0.831412  0.162362   0.214590          NaN   
       3  0.037326   0.155501  0.824411  0.158080   0.215319          NaN   
vina   1  0.067873   0.163426  0.789928  0.186328   0.196095          NaN   
       2  0.063301   0.161161  0.838570  0.194545   0.214547          NaN   
       3  0.062191   0.176506  0.837649  0.195335   0.214933          NaN   

           roc_auc  roc_log_auc        rp  rp_active   rp_inactive        rs  \
engine v                                                                       
dock   1  0.779697     0.320877  0.393128   0.209681 -7.286512e-16  0.141088   
       2  0.805416     0.323118  0.442243   0.231050 -4.366215e-16  0.150540   
       3  0.811157     0.335811  0.457017   0.254105 -1.439462e-15  0.152910   
dude   1  0.842279     0.317358  0.538024   0.286835  5.314050e-16  0.183406   
       2  0.867311     0.325330  0.606673   0.313199  3.183951e-16  0.196419   
       3  0.868832     0.334133  0.606251   0.315138 -1.615860e-16  0.193190   
vina   1  0.831231     0.295277  0.540249   0.286651 -6.586673e-16  0.228169   
       2  0.848905     0.294374  0.597505   0.311635 -2.947244e-16  0.237799   
       3  0.856707     0.309580  0.596713   0.309839 -4.476863e-16  0.239484   

          rs_active  rs_inactive  
engine v                          
dock   1   0.193364          NaN  
       2   0.212888          NaN  
       3   0.238350          NaN  
dude   1   0.278450          NaN  
       2   0.299183          NaN  
       3   0.300446          NaN  
vina   1   0.269998          NaN  
       2   0.299773          NaN  
       3   0.297580          NaN